<a href="https://colab.research.google.com/github/berbonilla/CCMACLRL_EXERCISES_COM221ML/blob/main/Excercise7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import seaborn as sns
import re
import os, types
import advertools as adv

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download("punkt")
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
splits = {'train': 'unique_train_dataset.csv', 'validation': 'unique_validation_dataset.csv', 'test': 'unique_test_dataset.csv'}

In [5]:
url = "https://github.com/stopwords-iso/stopwords-tl/blob/master/stopwords-tl.json"

In [6]:
df_train = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["train"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
df_validation = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["validation"])


In [8]:
df_test = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["test"])


In [9]:
data1 = df_train
data1

# converting to lowercase
data1["text"] = data1["text"].str.lower()

# Removing stopwords from the data
stop_words = adv.stopwords['tagalog']
data1["text"] = data1["text"].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

# applying lemmatization
wnl = WordNetLemmatizer()
data1["text"] = data1["text"].apply(lambda x: " ".join(wnl.lemmatize(word, "v") for word in x.split()))

data1

,text,label
0,presidential candidate mar roxas imply that th...,1
1,parang mali sumunod patalastas nescaf coffee b...,1
2,bet ko. pula kulay posas,1
3,[username] kakampink,0
4,parang tahimik pink about doc willie ong? no r...,1
...,...,...
21768,marcos talunan marcos magnanakaw,1
21769,grabe kayo kay binay ??????????,0
21770,[username] cnu ba naman hindimabibighani maam!...,0
21771,rt [username]: tabi tabi yung nagsasabing para...,1


In [10]:
vectorizer = CountVectorizer()
vectorized_data = vectorizer.fit_transform(data1["text"])


In [11]:
model = MultinomialNB(alpha=0.1)
model.fit(vectorized_data, data1['label'])

MultinomialNB(alpha=0.1)

In [12]:
  #evaluate model
  X_test = df_test["text"]
  Y_test = df_test["label"]

  predictions = model.predict(vectorizer.transform(X_test))
  accuracy = accuracy_score(Y_test, predictions)
  balanced_accuracy = balanced_accuracy_score(Y_test, predictions)
  precision = precision_score(Y_test, predictions)

  print("Accuracy:",round(100*accuracy,2),'%')
  print("Balanced accuracy:",round(100*balanced_accuracy,2),'%')
  print("Precision:", round(100*precision,2),'%')

Accuracy: 81.35 %
Balanced accuracy: 81.39 %
Precision: 77.38 %


In [21]:
new_text = pd.Series('Maraming salamat')
new_text_transform = vectorizer.transform(new_text)
predict = model.predict(new_text_transform)

# Interpret the prediction result
if predict == 1:
    print("The sentence is classified as hate speech.")
else:
    print("The sentence is classified as non-hate speech.")

The sentence is classified as non-hate speech.
